In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import unittest
import re
import numpy as np

In [2]:
# --- CONFIGURATION ---
#SHEET_PATH = "CM Data Collection - Virgo Cup (Responses) - exploded_all.csv"
SHEET_PATH = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR8Pa4QQVSNwepSe9dYnro3ZaVEpYQmBdZUzumuLL-U2IR3nKVh-_GbZeJHT2x9aCqnp7P-0hPm5Zd/pub?gid=221070242&single=true&output=csv"

# Forces charts to open in browser
pio.renderers.default = "browser"

## 1. Define Helper Functions

In [3]:
def find_column(df, keywords, case_sensitive=False):
    if df.empty: return None
    cols = df.columns.tolist()
    for col in cols:
        for key in keywords:
            if case_sensitive:
                if col == key: return col
            else:
                if col.lower() == key.lower(): return col
    clean_cols = df.columns.str.lower().str.replace(' ', '').str.replace('_', '').str.replace('-', '')
    for i, col in enumerate(clean_cols):
        for key in keywords:
            if key.lower() in col:
                return df.columns[i]
    return None

def clean_currency_numeric(series):
    return (series.astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)
            .str.replace(' ', '', regex=False)
            .str.replace('USD', '', regex=False)
            .str.replace('EUR', '', regex=False)
            .str.replace('++', '', regex=False)
            .str.replace('F2P', '0', regex=False)
            .str.split('-').str[0]
            .apply(pd.to_numeric, errors='coerce')
            .fillna(0))

def extract_races_count(series):
    def parse_races(text):
        text = str(text).lower()
        match = re.search(r'(\d+)\s*races', text)
        if match: return int(match.group(1))
        if text.isdigit(): return int(text)
        return 1 
    return series.apply(parse_races)

def parse_uma_details(series):
    # Enforce Title Case to ensure 'oguri' == 'Oguri'
    return series.astype(str).apply(lambda x: x.split(' - ')[0].strip().title())

def anonymize_players(df, metric='Calculated_WinRate', top_n=10):
    # Identify Top Performers by Total Wins + WinRate
    player_stats = df.groupby('Clean_IGN').agg({
        metric: 'mean',
        'Clean_Wins': 'sum',
        'Clean_Races': 'sum'
    }).reset_index()
    
    # Filter: Must have >20 races to be a "public" pro
    eligible_pros = player_stats[player_stats['Clean_Races'] >= 20]
    
    # SORT LOGIC: Prioritize Total Wins, THEN Win Rate
    top_players = eligible_pros.sort_values(['Clean_Wins', metric], ascending=[False, False]).head(top_n)['Clean_IGN'].tolist()
    
    df['Display_IGN'] = df['Clean_IGN'].apply(lambda x: x if x in top_players else "Anonymous Trainer")
    return df

## 2. Load & Process Data


In [4]:
try:
    df = pd.read_csv(SHEET_PATH)
    print(f"✅ Loaded Data: {len(df)} rows found.")

    # 1. Map Core Columns
    col_map = {
        'ign': find_column(df, ['ign', 'player']),
        'group': find_column(df, ['cmgroup', 'bracket']),
        'money': find_column(df, ['spent', 'eur/usd']),
        'uma': find_column(df, ['uma']),
        'style': find_column(df, ['style', 'running']),
        'wins': find_column(df, ['wins', 'victory']),
        'races_text': find_column(df, ['races', 'attempts']),
        'Round': find_column(df, ['Round'], case_sensitive=True), 
        'Day': find_column(df, ['Day'], case_sensitive=True),
        'runs_per_day': find_column(df, ['runsperday', 'howmanyruns'])
    }
    
    # 2. Clean Basic Data
    if col_map['money']: 
        df['Original_Spent'] = df[col_map['money']].fillna("Unknown")
        df['Sort_Money'] = clean_currency_numeric(df[col_map['money']])
    else: 
        df['Original_Spent'] = "Unknown"
        df['Sort_Money'] = 0.0

    if col_map['uma']: df['Clean_Uma'] = parse_uma_details(df[col_map['uma']])
    else: df['Clean_Uma'] = "Unknown"

    if col_map['wins']: df['Clean_Wins'] = pd.to_numeric(df[col_map['wins']], errors='coerce').fillna(0)
    else: df['Clean_Wins'] = 0
        
    if col_map['races_text']: df['Clean_Races'] = extract_races_count(df[col_map['races_text']])
    else: df['Clean_Races'] = 1

    # Win Rate Calc
    df['Calculated_WinRate'] = (df['Clean_Wins'] / df['Clean_Races']) * 100
    df.loc[df['Calculated_WinRate'] > 100, 'Calculated_WinRate'] = 100

    if col_map['group']: df['Clean_Group'] = df[col_map['group']].fillna("Unknown")
    else: df['Clean_Group'] = "Unknown"
    
    if col_map['ign']: df['Clean_IGN'] = df[col_map['ign']].fillna("Anonymous")
    else: df['Clean_IGN'] = "Anonymous"
    
    if col_map['style']: df['Clean_Style'] = df[col_map['style']].fillna("Unknown")
    else: df['Clean_Style'] = "Unknown"
    
    # Handle Round/Day
    if col_map['Round']: df['Round'] = df[col_map['Round']].fillna("Unknown")
    else: df['Round'] = "Unknown"
    
    if col_map['Day']: df['Day'] = df[col_map['Day']].fillna("Unknown")
    else: df['Day'] = "Unknown"

    # 3. Anonymize
    df = anonymize_players(df)
    
    print("✅ Data Processing Complete!")

except Exception as e:
    print(f"❌ Data Error: {e}")
    df = pd.DataFrame()

✅ Loaded Data: 6115 rows found.
✅ Data Processing Complete!


 ## 3. Advanced Analysis: Team Reconstruction

In [5]:
if not df.empty:
    # 1. Group by Session to rebuild Teams
    # Sorting Umas ensures [A, B, C] is same as [C, B, A]
    team_df = df.groupby(['Clean_IGN', 'Display_IGN', 'Clean_Group', 'Round', 'Day', 'Original_Spent', 'Sort_Money']).agg({
        'Clean_Uma': lambda x: sorted(list(x)), 
        'Clean_Style': lambda x: list(x),       
        'Calculated_WinRate': 'mean',           
        'Clean_Races': 'mean',
        'Clean_Wins': 'mean'
    }).reset_index()

    # 2. Filter: Must have exactly 3 Umas
    team_df['Uma_Count'] = team_df['Clean_Uma'].apply(len)
    team_df = team_df[team_df['Uma_Count'] == 3]

    # 3. Create "Team Signature"
    team_df['Team_Comp'] = team_df['Clean_Uma'].apply(lambda x: ", ".join(x))
    
    # 4. Filter: Remove Comps with <= 7 entries
    comp_counts = team_df['Team_Comp'].value_counts()
    valid_comps = comp_counts[comp_counts > 7].index.tolist()
    
    filtered_team_df = team_df[team_df['Team_Comp'].isin(valid_comps)]
    
    # 5. RUNAWAY ANALYSIS (Fixed Logic)
    def check_for_runaway(style_list):
        target_terms = ['Runaway', 'Runner', 'Escape', 'Oonige', 'Great Escape']
        for s in style_list:
            s_clean = str(s).strip()
            s_lower = s_clean.lower()
            if any(t.lower() in s_lower for t in target_terms):
                if "front" in s_lower: continue
                return True
        return False

    team_df['Has_Runaway'] = team_df['Clean_Style'].apply(check_for_runaway)

## 4. Advanced Plots (Beautified)


In [6]:
# --- PLOT 1: Money vs Win Rate (BOX PLOT) ---
if not team_df.empty:
    print("📊 Plotting Money Box Plot...")
    team_df = team_df.sort_values('Sort_Money')
    fig_money = px.box(team_df, x='Original_Spent', y='Calculated_WinRate', color='Original_Spent', points="all", title="Win Rate Distribution by Spending Tier", template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Bold)
    fig_money.update_layout(showlegend=False)
    fig_money.show()

📊 Plotting Money Box Plot...


In [7]:
# --- PLOT 2: Ideal Team Comps (Colorful Bar) ---
if not filtered_team_df.empty:
    comp_stats = filtered_team_df.groupby('Team_Comp').agg({
        'Calculated_WinRate': 'mean', 
        'Clean_Races': 'count'
    }).reset_index().rename(columns={'Clean_Races': 'Usage Count'})
    
    fig_comps = px.bar(
        comp_stats.sort_values('Calculated_WinRate', ascending=False).head(15),
        x='Calculated_WinRate',
        y='Team_Comp',
        orientation='h',
        color='Calculated_WinRate',
        color_continuous_scale='Plasma',
        title="Ideal Team Compositions (Min. 7 Uses)",
        text='Usage Count', 
        hover_data=['Usage Count'],
        labels={'Calculated_WinRate': 'Avg Win Rate (%)', 'Team_Comp': '', 'Usage Count': 'Entries'},
        template='plotly_dark'
    )
    fig_comps.update_layout(yaxis={'categoryorder':'total ascending'})
    fig_comps.show()

In [8]:
# ## Plot: Individual Uma Tier List
# This aggregates stats by individual Uma, regardless of their team.
if not df.empty:
    uma_stats = df.groupby('Clean_Uma').agg({
        'Calculated_WinRate': 'mean',
        'Clean_Races': 'count'
    }).reset_index()
    
    # Filter: Remove Umas with very low usage (< 10 runs)
    uma_stats = uma_stats[uma_stats['Clean_Races'] >= 10]
    
    fig_uma = px.bar(
        uma_stats.sort_values('Calculated_WinRate', ascending=False).head(15),
        x='Calculated_WinRate',
        y='Clean_Uma',
        orientation='h',
        color='Calculated_WinRate',
        color_continuous_scale='Viridis',
        title="Individual Uma Performance (Min. 10 Runs)",
        text='Clean_Races',
        labels={'Calculated_WinRate': 'Avg Win Rate (%)', 'Clean_Uma': 'Character', 'Clean_Races': 'Runs'},
        template='plotly_dark'
    )
    fig_uma.update_layout(yaxis={'categoryorder':'total ascending'})
    fig_uma.show()

In [9]:
# ## Plot: Win Rate Trend by Round/Day
if 'Round' in df.columns and 'Day' in df.columns:
    trend_df = team_df.groupby(['Round', 'Day']).agg({
        'Calculated_WinRate': 'mean',
        'Clean_Races': 'count'
    }).reset_index()
    
    # Create a sortable session ID
    trend_df['Session'] = trend_df['Round'] + " " + trend_df['Day']
    
    fig_trend = px.line(
        trend_df,
        x='Session',
        y='Calculated_WinRate',
        title="Average Win Rate by Session (Meta Evolution)",
        markers=True,
        template='plotly_dark'
    )
    fig_trend.update_layout(yaxis_title="Avg Win Rate (%)")
    fig_trend.show()

In [10]:
# ## Plot: Simplified Leaderboard
if not team_df.empty:
    named_teams = team_df[team_df['Display_IGN'] != "Anonymous Trainer"].copy()
    
    # Aggregate All-Time Stats
    leaderboard = named_teams.groupby(['Display_IGN', 'Team_Comp']).agg({
        'Clean_Wins': 'sum',
        'Clean_Races': 'sum'
    }).reset_index()
    
    leaderboard['Global_WinRate'] = (leaderboard['Clean_Wins'] / leaderboard['Clean_Races']) * 100
    
    # Strict Filter: Must have played > 15 races
    leaderboard = leaderboard[leaderboard['Clean_Races'] >= 15]
    
    # Sort by Win Rate
    top_leaders = leaderboard.sort_values('Global_WinRate', ascending=False).head(10)
    top_leaders['Label'] = top_leaders['Display_IGN'] + " (" + top_leaders['Team_Comp'] + ")"
    
    fig_leader = px.bar(
        top_leaders,
        x='Global_WinRate',
        y='Label',
        orientation='h',
        color='Global_WinRate',
        title="Top 10 Trainers (Min. 15 Races)",
        text='Clean_Wins', # Show Total Wins on the bar
        labels={'Global_WinRate': 'Win Rate (%)', 'Label': 'Trainer', 'Clean_Wins': 'Total Wins'},
        template='plotly_dark',
        color_continuous_scale='Turbo'
    )
    fig_leader.update_layout(yaxis={'categoryorder':'total ascending'})
    fig_leader.update_traces(texttemplate='Wins: %{text}', textposition='inside')
    fig_leader.show()

In [11]:
# ## Plot: Strategy Analysis
if 'Clean_Style' in df.columns:
    # 1. STYLE ANALYSIS
    def standardize_style(style):
        s = str(style).lower().strip()
        if 'front' in s or 'leader' in s: return 'Front Runner'
        if 'pace' in s or 'betweener' in s: return 'Pace Chaser'
        if 'late' in s: return 'Late Surger'
        if 'end' in s or 'closer' in s: return 'End Closer'
        if 'run' in s or 'escape' in s or 'oonige' in s: return 'Runaway'
        return 'Unknown'

    style_df = df.copy()
    style_df['Standard_Style'] = style_df['Clean_Style'].apply(standardize_style)
    style_stats = style_df.groupby('Standard_Style').agg({'Calculated_WinRate': 'mean', 'Clean_Races': 'count'}).reset_index()
    style_stats = style_stats[(style_stats['Clean_Races'] > 20) & (style_stats['Standard_Style'] != 'Unknown')]
    
    desired_order = ['Runaway', 'Front Runner', 'Pace Chaser', 'Late Surger', 'End Closer']
    
    fig_style = px.bar(
        style_stats,
        x='Calculated_WinRate',
        y='Standard_Style',
        orientation='h',
        color='Calculated_WinRate',
        title="Performance by Running Style",
        text='Calculated_WinRate',
        template='plotly_dark',
        color_continuous_scale='Viridis'
    )
    fig_style.update_layout(yaxis={'categoryorder':'array', 'categoryarray': desired_order[::-1]})
    fig_style.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig_style.show()

    # 2. RUNAWAY IMPACT
    def check_for_runaway(style_list):
        target_terms = ['Runaway', 'Runner', 'Escape', 'Oonige', 'Great Escape']
        for s in style_list:
            s_clean = str(s).strip()
            s_lower = s_clean.lower()
            if any(t.lower() in s_lower for t in target_terms):
                if "front" in s_lower: continue 
                return True
        return False
    
    team_df['Has_Runaway'] = team_df['Clean_Style'].apply(check_for_runaway)
    runner_stats = team_df.groupby('Has_Runaway')['Calculated_WinRate'].mean().reset_index()
    runner_stats['Strategy'] = runner_stats['Has_Runaway'].map({True: 'With Runaway (Nigeru)', False: 'No Runaway'})
    
    fig_runner = px.bar(
        runner_stats, 
        x='Strategy', 
        y='Calculated_WinRate', 
        color='Strategy', 
        template='plotly_dark', 
        title="Impact of having a Runaway in Team",
        text='Calculated_WinRate',
        color_discrete_sequence=['#00CC96', '#EF553B']
    )
    fig_runner.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig_runner.show()

In [12]:
# --- PLOT 5: Runaway Impact ---
if 'Has_Runaway' in team_df.columns:
    print("📊 Plotting Runaway Impact...")
    runner_stats = team_df.groupby('Has_Runaway')['Calculated_WinRate'].mean().reset_index()
    runner_stats['Strategy'] = runner_stats['Has_Runaway'].map({True: 'With Runaway (Nigeru)', False: 'No Runaway'})
    
    fig_runner = px.bar(
        runner_stats, 
        x='Strategy', 
        y='Calculated_WinRate', 
        color='Strategy', 
        template='plotly_dark', 
        title="Impact of having a Runaway (Oonige) in Team",
        color_discrete_sequence=['#00CC96', '#EF553B']
    )
    fig_runner.show()

📊 Plotting Runaway Impact...


In [15]:

# Dynamically find all card columns to ensure R Riko vs SSR Riko are distinct.
card_map = {}
for c in df.columns:
    if "Card Status" in c:
        # Extract name from brackets: "... [SSR Riko]" -> "SSR Riko"
        card_name = c.split('[')[-1].replace(']', '').strip()
        card_map[card_name] = c

if card_map:
    # Sort cards alphabetically for the dropdown
    available_cards = sorted(list(card_map.keys()))
    
    # Simulate selection (Default to first one found)
    # In the real app, this will be st.selectbox
    target_card_name = available_cards[4] 
    target_col = card_map[target_card_name]
    
    print(f"📊 Plotting Card Impact for: {target_card_name}")
    
    # Use session-unique data to avoid double counting players
    card_stats = df.drop_duplicates(subset=['Clean_IGN', 'Round', 'Day']).groupby(target_col)['Calculated_WinRate'].mean().reset_index()
    
    fig_card = px.bar(
        card_stats,
        x=target_col,
        y='Calculated_WinRate',
        color='Calculated_WinRate', 
        color_continuous_scale='Bluered', 
        title=f"Win Rate by {target_card_name} Status",
        labels={target_col: "Limit Break Status", 'Calculated_WinRate': 'Win Rate (%)'},
        template='plotly_dark'
    )
    fig_card.show()

📊 Plotting Card Impact for: SSR Riko Kashimoto


{'Kitasan Black': 'Card Status in Account (Non-Borrow) [Kitasan Black]', 'Super Creek': 'Card Status in Account (Non-Borrow) [Super Creek]', 'Fine Motion': 'Card Status in Account (Non-Borrow) [Fine Motion]', 'SSR Riko Kashimoto': 'Card Status in Account (Non-Borrow) [SSR Riko Kashimoto]', 'R Riko Kashimoto': 'Card Status in Account (Non-Borrow) [R Riko Kashimoto]', 'Rice Shower Power': 'Card Status in Account (Non-Borrow) [Rice Shower Power ]'}


## 5. 🚀 EXPORT TO DASHBOARD.PY